# Реализация Digital Signature Algorithm

**Digital Signature Algorithm (DSA — алгоритм цифровой подписи)** — криптографический алгоритм с использованием закрытого ключа (из пары ключей: <открытый; закрытый>) для создания электронной подписи, но не для шифрования (в отличие от RSA и схемы Эль-Гамаля). 

Подпись создается секретно (закрытым ключом), но может быть публично проверена (открытым ключом). Это означает, что только один субъект может создать подпись сообщения, но любой может проверить её корректность.

Алгоритм основан на вычислительной сложности взятия логарифмов в конечных полях.

### Иллюстрация алгоритма:
<img src="https://upload.wikimedia.org/wikipedia/commons/a/a9/Dsa_workflow_rus.png">

### Алгоритм решения

1. Выбрать $L$-битный прайм-$р$, так, чтобы он удовлетворял следующему:
    - $512 <= L <= 1024$
    - $64$ деления $L$
    - $p-1$ имеет 160-битный прайм-фактор, например $q$.
2. Выбрать $g$, который принадлежит к группе $Zp^*$ так, чтобы порядок $g$ был $q$
3. Выбрать число равномерно случайным образом из множества ${2,3,...,q-1}$
4. Вычислить $h = g^a \ mod \ p$
5. Ключ проверки = $(p,q,g,h)$
6. Ключ подписи = $a$


### Процесс подписи

В качестве входа является документ $F$, ключ верификации и ключ подписи. Выводом является подпись для $F$ в ```data/signature.txt```.

1. Выбрать случайный элемент $r: 1 <= r <= q-1$
2. Вычислить $C_1 = (g^r \ mod \ p) \ mod \ q$
3. Вычислить $C_2 = (int(SHA1(F))) + a  C_1)r^{-1} \ mod \ q$
4. Если $C_1 = 0$ или $C_2 = 0$, следует выбрать новое случайное значение r и пересчитать $C_1$, $C_2$
5. Вывести $(C1, C2)$ в качестве подписи для документа $F$


### Процесс верификации 

В качестве входа является пара документ и подпись $(F, (C_1, C_2))$ и VerKey (p, q, g, h).

1. Вычислить $t_1 = int(SHA1(F)) * C_2^{-1} \ mod \ q$
2. Вычислить $t_2 = C_1 * C_2^{-1} \ mod \ q$
3. Если $(g^{t_1} * h^{t_2} \ mod \ p) \ mod \ q = C_1$, то подпись действительна, иначе подпись недействительна


# Демонстрация решения

In [1]:
import os

from src.key import key_generation
from src.sign import sign, verification

## Чтение документа для подписи

In [2]:
file_name = 'doc.txt'

### Проверка файла и директории

In [3]:
if not os.path.isdir('data'):
    os.mkdir('data')
    
if not os.path.isfile(f'data/{file_name}'):
    print(f'Create simple message for {file_name} file...')
    simple_message = 'Hello world!'
    file = open(f"data/{file_name}", "w")
    file.write(simple_message)
    file.write("\n")
    file.close()

### Содержание документа

In [4]:
with open(f'data/{file_name}', 'r') as file:
    message = file.read()
    print(message)

Delete before read.
It's serious question - would you like a tomate juice?


## Создадим ключ

In [5]:
key_generation()

Computing key values, please wait...
p =  143730415632815857251629835921439553316475944667997939180132860490518584771784437837160858489650484680504785454990117075520236795545801076232695634674740427515010559001531257
q =  870984545871652160342030840793606137846256289691
g =  7295764318237127076587627316436346790042269759246857213493612493100266684970334056007248574803855884194420070605525219073718132552921026211432647416831040021536755348522715
h =  136546672615172778997646061033112686861266525328520068923266036596942269403504994335738550244570007324762552506838496789394899011515076980725533507100723910013147919945846992
a =  661102633699384079157598972043919091075614232169
Verification key stored at data/key.txt and secret key stored at data/secret_key.txt


## Создадим подпись на основе ключа

In [6]:
sign(file_name)

Signing the file...
hash =  729231516461238860167057583315706068465413541073
c1 =  255789384869263148990177519013882297399698464779
c2 =  375453649704671206983254924308149892403553096862
cipher stored at signature.txt


## Проверка подписи

In [7]:
verification(file_name)

Verification process...
c1 =  255789384869263148990177519013882297399698464779
c2 =  375453649704671206983254924308149892403553096862
hash =  729231516461238860167057583315706068465413541073
Valid signature
